Day4_3_visualization.ipynb 에서 단어를 custom_tokenizer를 한 다음, 문서 벡터로 표현하여 그대로 시각화 방법을 이용하여 2차원 벡터로 학습하였습니다. 

이 경우에는 말미에 언급한 것처럼 노이즈한 결과가 나오기도 합니다. 시각화는 대상을 좋은 벡터로 표현한 다음에 (representation learning), 이를 저차원의 벡터로 다시 한 번 압축해야 합니다. 

대부분의 단어/문서 시각화는 Word2Vec / Doc2Vec / Glove / RNN / RBM 등으로 질 좋은 고차원 벡터의 representation을 얻은 뒤, 이를 다시 한 번 2차원 벡터로 압축하고 있습니다. 한 번에 풀리는 건 없습니다. 특히나 시각화와 같이 노이즈에 민감한 작업에서는 더 그러합니다. 

이번에는 4_3과 동일한 문서에 대하여 토크나이징을 하지 않은 체로 어절을 그대로 Word2Vec 학습을 하겠습니다. 데이터의 숫자가 정말로 풍부하고, 모델이 수많은 어절을 그대로 학습할 수 있을만큼의 하드웨어라면, 어절을 그대로 Word2Vec 학습하여도 그 결과는 충분히 쓸만합니다만, 그 목적이 다를 수 있습니다. 아래에서 그 예시를 살펴보겠습니다. 

In [7]:
corpus_fname = '../../../data/corpus_10days/news/2016-10-24_article_all_normed.txt'
word2vec_fname = '../../../data/corpus_10days/models/2016-10-24_news_word2vec_model.pkl'

TRAIN_WORD2VEC = True

import sys
sys.path.append('../mypy/')

from corpus import Corpus
corpus = Corpus(corpus_fname, iter_sent=True)
len(corpus)

213087

doublespace line spliter이기 때문에 Word2VecCorpus는 '  '를 기준으로 doc을 sents로 나눈 뒤, 각 sent를 다시 한 번 split()하여 word list로 yield 합니다. 

In [5]:
import os
    
class Word2VecCorpus:
    
    def __init__(self, fname):
        self.fname = fname
        if not os.path.isfile(fname):
            print('File not found: %s' % fname)
        
    def __iter__(self):
        with open(self.fname, encoding='utf-8') as f:
            for doc in f:
                for sent in doc.strip().split('  '):
                    sent = sent.strip()
                    if not sent:
                        continue
                    yield sent.split()
                
                
word2vec_corpus = Word2VecCorpus(corpus_fname)

for num_sent, sent in enumerate(word2vec_corpus):
    if num_sent > 5: break
    print(sent)

['의원', '60명', '내무장관에게', '서한', '철거', '때', '어린이', '안전', '신경', '써야']
['보호자', '없는', '아동', '난민', '70명은', '영국에', '도착']
['22일', '경찰에', '돌', '던지는', '칼레', '난민', '연합뉴스']
['파리', '런던', '연합뉴스', '박성진', '황정우', '특파원', '프랑스', '칼레', '난민촌', '철거를', '이틀', '앞둔', '22일', '현지시간', '난민촌', '주변에서', '현지', '경찰과', '난민이', '충돌했다고', '현지', '프랑스앵포가', '보도했다']
['난민촌', '철거에', '반대하는', '난민', '50명가량은', '경찰을', '향해', '유리병과', '돌을', '던졌으며', '경찰은', '연막탄을', '쏘면서', '이들을', '해산했다']
['이', '과정에서', '별다른', '부상자는', '나오지', '않았다']


In [17]:
from gensim.models import Word2Vec
import pickle

if TRAIN_WORD2VEC:
    word2vec_model = Word2Vec(word2vec_corpus, min_count=30)
    with open(word2vec_fname, 'wb') as f:
        pickle.dump(word2vec_model, f)
        
else:
    with open(word2vec_fname, 'rb') as f:
        word2vec_model = pickle.load(f)

gensim.__version__ = 0.13.x 기준으로, Word2Vec.syn0에는 단어 벡터가 저장되어 있습니다. 

In [18]:
len(word2vec_model.syn0)

18329

In [20]:
test_words = ['김무성', 
              '박근혜', 
              '문재인', 
              '국방부', 
              '정부', 
              '국정원', 
              '대통령', 
              '축구', 
              '외교', 
              '정책', 
              '군대', 
              '미국', 
              '일본', 
              '중국'
             ]

어절만 이용하여 학습했음에도 불구하고, 김무성의 유사어로 정치인들이 등장함을 볼 수 있습니다. 이는 뉴스 코퍼스에서는 정치인의 이름만 적고 띄어쓰기를 하는 경우도 충분하기 때문입니다. 

하지만 '김무성 - 대표도'와 같은 유사 어절이 등장한 것은, 뉴스에서 '김무성'으로만 적는 경우와 '김무성 대표도'로 적는 경우가 혼재되어 있어서, '대표도'의 문맥이 '김무성'의 문맥과 유사하였기 때문입니다. 

비슷한 의미로, 박근혜 - 박 (6251)의 경우에는 '박근혜 대통령', '박 대통령'이 번갈아가며 이용되었기 때문입니다. 


그러나, 국정원의 경우에는 총 42번 나왔으며, min_count=30으로 하였기 때문에 18000여개의 단어 중에서는 infrequent 한 편에 속합니다. 그리고 이 때의 유사 어절들은 문맥상 잘 어울리지 않는 어절들입니다. 

    Vocab(count:42, index:12945, sample_int:4294967296)
    국정원 - 단국대 (33) (0.774)
    국정원 - 대우학원 (34) (0.726)
    국정원 - 김형수 (92) (0.722)
    국정원 - 인터뷰가 (36) (0.717)
    국정원 - 입 (59) (0.711)
    국정원 - 정동구 (31) (0.709)
    국정원 - 정유라씨가 (41) (0.705)
    국정원 - 취재진이 (33) (0.703)
    국정원 - 크리스 (33) (0.696)
    국정원 - 이사장을 (81) (0.696)
    
Word2Vec에서는 infrequent 한 단어(어절)들의 유사 단어(어절)들은 infrequent 한 경향이 있습니다. 빈도수가 충분하지 않을 경우에는 학습이 잘 되지 않는 것이라고 해석할 수 있습니다. 

어절을 그대로 학습할 경우에는, 아래와 같이 대통령의 유사어절로 '대통령 + 조사'의 어절들이 자주 등장합니다. 
    
    Seed word = 대통령
    Vocab(count:3411, index:89, sample_int:4294967296)
    대통령 - 대통령의 (3074) (0.809)
    대통령 - 대통령을 (287) (0.784)
    대통령 - 대통령과 (437) (0.784)
    대통령 - 대통령에 (255) (0.723)
    대통령 - 대통령은 (2852) (0.716)
    대통령 - 대통령에게 (203) (0.708)
    대통령 - 대통령도 (125) (0.704)
    대통령 - 대통령이다 (54) (0.698)
    대통령 - 대통령이 (5054) (0.698)
    대통령 - 정부에서 (76) (0.687)
    
우리가 원하는 것이 Language model을 학습하는 것이라면, 이 결과는 유용합니다. 하지만, 우리가 문맥적으로 유사한 단어들을 찾기 위하여 Word2Vec을 학습하는 것이 목적이었다면, 어절을 그대로 학습하는 것보다 토크나이징을 한 뒤, 이를 이용하여 학습해야 합니다. 

In [23]:
for word in test_words:
    if (word in word2vec_model.vocab) == False:
        continue
        
    print('\n\nSeed word = %s' % word)
    print(word2vec_model.vocab[word])
    for similar_word, sim in word2vec_model.most_similar(word):
        print('%s - %s (%d) (%.3f)' % (word, 
                                       similar_word, 
                                       word2vec_model.vocab[similar_word].count, 
                                       sim))



Seed word = 김무성
Vocab(count:332, index:1413, sample_int:4294967296)
김무성 - 김종인 (165) (0.935)
김무성 - 오세훈 (72) (0.920)
김무성 - 안철수 (345) (0.916)
김무성 - 손학규 (270) (0.895)
김무성 - 대표도 (106) (0.887)
김무성 - 유승민 (160) (0.877)
김무성 - 천정배 (36) (0.866)
김무성 - 최경환 (43) (0.866)
김무성 - 정의화 (35) (0.850)
김무성 - 김부겸 (100) (0.819)


Seed word = 박근혜
Vocab(count:4420, index:62, sample_int:4294967296)
박근혜 - 이승만 (36) (0.753)
박근혜 - 박 (6251) (0.735)
박근혜 - 마두로 (46) (0.712)
박근혜 - 박정희 (216) (0.708)
박근혜 - 국정과 (37) (0.701)
박근혜 - 뒤늦은 (30) (0.688)
박근혜 - 민생에 (38) (0.679)
박근혜 - 나서달라고 (59) (0.655)
박근혜 - 두테르테 (218) (0.654)
박근혜 - 요청했고 (73) (0.653)


Seed word = 문재인
Vocab(count:940, index:392, sample_int:4294967296)
문재인 - 손학규 (270) (0.883)
문재인 - 비서실장이 (97) (0.869)
문재인 - 문 (932) (0.855)
문재인 - 안철수 (345) (0.851)
문재인 - 김종인 (165) (0.850)
문재인 - 장관의 (162) (0.803)
문재인 - 송민순 (663) (0.798)
문재인 - 대표가 (1013) (0.789)
문재인 - 급급한 (39) (0.788)
문재인 - 김무성 (332) (0.787)


Seed word = 국방부
Vocab(count:247, index:2008, sample_int:4294967296)
국방부 - 외교부 (